## Агрегирование данных и групповые операции

Разбиение набора данных на группы и применение некоторой функции к каждой группе, будь то в целях агрегирования или преобразования, зачастую является одной из важнейших операций анализа данных. После загрузки, слияния и подготовки набора данных обычно вычисляют статистику по группам или, возможно, сводные таблицы для построения отчета или визуализации. В библиотеке pandas имеется гибкий и быстрый механизм groupby, который позволяет формировать продольные и поперечные срезы, а также агрегировать наборы данных естественным образом. 


Одна из причин популярности реляционных баз данных и языка SQL - простота соединения, фильтрации, преобразования и агрегирования данных. Однако в том, что касается групповых операций, языки запросов типа SQL несколько ограничены. Выразительность и мощь языка Python и библиотеки pandas позволяют выполнять гораздо более сложные групповые операции с помощью функций, принимающих произвольный объект pandas или массив NumPy.



### Механизм GroupBy

DataFrame можно группировать как по строкам (axis = 0), так и по столбцам (axis = 1). Затем к каждой группе применяется некоторая функция, которая порождает новое значение. Наконец, результаты применения всех функций объединяются в результирующий
объект.

Ключи группировки могут задаваться по-разному и необязательно должны быть одного типа:

- список или массив значений той же длины, что ось, по которой производится группировка;
- значение, определяющее имя столбца объекта DataFrame;
- словарь или объект Series, определяющий соответствие между значениями на оси группировки и именами групп;
- функция, которой передается индекс оси или отдельные метки из этого индекса.

Последние три метода - просто различные способы порождения массива значений, используемого далее для разделения объекта на группы. 

Для начала рассмотрим очень простой табличный набор данных, представленный в виде объекта DataFrame:


In [ ]:
from pandas import *
import numpy as np

In [ ]:
df = DataFrame({'key1': ['а', 'а', 'b', 'b', 'а'], 
                'key2': [ 'one', 'two', 'one', 'two', 'one'],
                'data1': np.random.randn(5), 
                'data2': np.random.randn(5)})

df

---
Пусть требуется вычислить среднее по столбцу data1, используя метки групп в столбце key1. Сделать это можно несколькими способами. Первый - взять столбец data1 и вызвать метод groupby, передав ему столбец (объект Series) key1:



---


In [ ]:
grouped = df['data1'].groupby(df['key1'])
grouped

---
Переменная grouped - это объект GroupBy. Пока что он не вычислил ничего, кроме промежуточных данных о групповом ключе df [ ‘key1’]. Идея в том, что
этот объект хранит всю информацию, необходимую для последующего применения некоторой операции к каждой группе. Например, чтобы вычислить средние по группам, мы можем вызвать метод mean объекта GroupBy:

---



In [ ]:
grouped.mean()

---

В результате вызова GroupBy данные (объект Series) агрегированы по групповому ключу, и в результате создан новый объект Series, индексированный уникальными значениями в столбце key1. Получившийся индекс назван 'key1', потому что так назывался столбец df ['key1'] объекта DataFrame.
Если бы мы передали несколько массивов в виде списка, то получили бы другой результат:




In [ ]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means


### Обход групп

Объект GroupBy поддерживает итерирование, в результате которого генерируется последовательность 2-кортежей, содержащих имя группы и блок данных.
Рассмотрим следующий небольшой набор данных:



In [ ]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

В случае нескольких ключей первым элементом кортежа будет кортеж, содержащий значения ключей:

In [ ]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print (k1, k2)
    print (group)

Разумеется, только вам решать, что делать с блоками данных. Возможно, пригодится следующий однострочный код, который строит словарь блоков:

In [ ]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

По умолчанию метод groupby группирует по оси axis=0, но можно задать любую другую ось. Например, в нашем примере столбцы объекта df можно было бы сгруппировать по dtype:

In [ ]:
print(df.dtypes)
grouped = df.groupby(df.dtypes, axis=1)
dict(list(grouped))

### Выборка столбца или подмножества столбцов

Доступ по индексу к объекту GroupBy, полученному группировкой объекта DataFrame путем задания имени столбца или массива имен столбцов, имеет тот же эффект, что выборка этих столбцов для агрегирования. Это означает, что

df.groupby('key1')['data1']

df.groupby('key1' )[['data2']]

- в точности то же самое, что:

df['data1'].groupby(df['keyl'])

df[['data2']].groupby (df['keyl'])


Большие наборы данных обычно желательно агрегировать лишь по немногим столбцам. Например, чтобы в приведенном выше примере вычислить среднее только по столбцу data2 и получить результат в виде DataFrame, можно было бы написать:

In [ ]:
df.groupby(['key1', 'key2'])[('data2')].mean()

В результате этой операции доступа по индексу возвращается сгруппированный DataFrame, если передан список или массив, или сгруппированный Series, если передано только одно имя столбца: 

In [ ]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped.mean()

### Группировка с помощью словарей и объектов Series

Информацию о группировке можно передавать не только в виде массива. Рассмотрим еще один объект DataFrame:

In [ ]:
people = DataFrame(np.random.randn(5, 5),
                   columns=['a', 'b', 'с', 'd', 'е'],
                   index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people

In [ ]:
people.loc[2:3, ('b', 'с')] = np.nan  # Добавим несколько пустух значений
people


Теперь предположим, что имеется соответствие между столбцами и группами, и нужно просуммировать столбцы для каждой группы:

In [ ]:
mapping = {'a':'red', 'b':'red', 'с':'blue', 'd':'blue', 'е':'red' , 'f':'orange'} 
mapping

Из этого словаря нетрудно построить массив и передать его groupby, но можно вместо этого передать и сам словарь:

In [ ]:
by_column = people.groupby(mapping, axis=1)
by_column.sum()

То же самое относится и к объекту Series, который можно рассматривать как отображение фиксированного размера. Когда в рассмотренных выше примерах применялся объект Series для задания групповых ключей, pandas на самом деле проверяла, что индекс Series выровнен с осью, по которой производится группировка:

In [ ]:
map_series = Series(mapping)
map_series

In [ ]:
people

In [ ]:
people.groupby(map_series, axis=1).count()

### Группировка с помощью функций

Использование функции Python - более абстрактный способ определения соответствия групп по сравнению со словарями или объектами Series. Функция, переданная в качестве группового ключа, будет вызвана по одному разу для каждого значения в индексе, а возвращенные ей значения станут именами групп. Конкретно, рассмотрим пример объекта DataFrame из предыдущего раздела, где значениями индекса являются имена людей. Пусть требуется сгруппировать по длине имени; можно было бы вычислить массив длин строк, но лучше вместо этого просто передать функцию len:

In [ ]:
people.groupby(len).sum()

### Агрегирование данных

Под агрегированием мы будем понимать любое преобразование данных, которое порождает скалярные значения из массивов. В примерах выше вы встречали несколько таких преобразований: mean, count, min и sum.


Для иллюстрации более сложных возможностей агрегирования рассмотрим набор данных о ресторанных чаевых. Впервые он был приведен в книге Брайана и Смита по экономической статистике 1995 года.

In [ ]:
tips = read_csv('tips.csv')
tips[:3]

In [ ]:
# Добавим величину чаевых в виде процента от суммы счета

tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

#### Применение функций, зависящих от столбца, и нескольких функций

Для агрегирования объекта Series или всех столбцов объекта DataFrame достаточно воспользоваться методом aggregate, передав ему
требуемую функцию, или вызвать метод mean, std и им подобный. Однако иногда нужно использовать разные функции в зависимости от столбца или сразу несколько функций.

In [ ]:
# Для начала сгруппируем столбец tips по значениям sex и smoker
grouped = tips.groupby(['sex', 'smoker'])

In [ ]:
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

Если вместо этого передать список функций или имен функций, то будет возвращен объект DataFrame, в котором имена столбцов совпадают с именами функций:

In [ ]:
grouped_pct.agg(['mean','std'])

Совершенно необязательно соглашаться с именами столбцов, предложенными объектом GroupBy; в частности, все лямбда-функции называются `<lambda>`, поэтому различить их затруднительно. Поэтому если передать список кортежей вида **(name, function)**, то в качестве имени столбца DataFrame будет взят первый элемент кортежа (можно считать, что список 2-кортежей - упорядоченное отображение):

In [ ]:
grouped_pct.agg ([('foo', 'mean'), ('bar', np.std)])

В случае DataFrame диапазон возможностей шире, поскольку можно задавать список функций, применяемых ко всем столбцам, или разные функции для разных столбцов. Допустим, нам нужно вычислить три одинаковых статистики для столбцов tip_pct и total_bill:

In [ ]:
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill' ].agg(functions)
result

В результирующем DataFrame имеются иерархические столбцы - точно так же, как было бы, если бы мы агрегировали каждый столбец по отдельности, а потом склеили результаты с помощью метода concat, передав ему имена столбцов в качестве аргумента keys:

In [ ]:
result ['tip_pct']

Предположим далее, что требуется применить потенциально различные функции к одному или нескольким столбцам. Делается это путем передачи методу agg словаря, который содержит отображение имен столбцов на любой из рассмотренных выше объектов, задающих функции:

In [ ]:
grouped.agg({ 'tip': np.max, 
             'size': 'sum'})

In [ ]:
grouped.agg({'tip_pct': ['min', 'max', 'mean', 'std'], 'size' : 'sum'})

Объект DataFrame будет содержать иерархические столбцы, только если хотя бы к одному столбцу было применено несколько функций.

#### Возврат агрегированных данных в «неиндексированном» виде

Во всех рассмотренных выше примерах агрегированные данные сопровождались индексом, иногда иерархическим, составленным из уникальных встретившихся комбинаций групповых ключей. Такое поведение не всегда желательно, поэтому его можно подавить, передав методу groupby параметр as_index= False:

In [ ]:
tips.groupby(['sex', 'smoker'], as_index = False).mean()

Разумеется, для получения данных в таком формате всегда можно вызвать метод reset_index результата.

## Групповые операции и преобразования

Агрегирование - лишь одна из групповых операций. Это частный случай более общего класса преобразований, в котором применяемая функция редуцирует одномерный массив в скалярное значение.

Методы **transform** и **apply**, позволяют выполнять групповые операции других видов. 

Предположим, что требуется добавить столбец в объект DataFrame, содержащий групповые средние для каждого индекса. Для этого можно было сначала выполнить агрегирование, а затем слияние:


In [ ]:
df

In [ ]:
k1_means = df.groupby('key1').mean().add_prefix('mean_')
k1_means

In [ ]:
merge(df, k1_means, left_on='key1', right_index=True)

Этот способ работает, но ему недостает гибкости. Данную операцию можно рассматривать как преобразование двух столбцов с помощью функции np.mean. Рассмотрим еще раз объект DataFrame people, встречавшийся выше, и воспользуемся методом **transform** oбъeктa GroupBy:

In [ ]:
people

In [ ]:
key = ['one', 'two', 'one', 'two', 'one']

In [ ]:
people.groupby(key).mean()

In [ ]:
people.groupby(key).transform(np.mean)

Как легко догадаться, **transform** применяет функцию к каждой группе, а затем помещает результаты в нужные места. Если каждая группа порождает скалярное значение, то оно будет распространено (уложено). Но допустим, что требуется вычесть среднее значение из каждой группы. Для этого напишем функцию demean и передадим ее методу transform:

In [ ]:
def demean(arr):
    return arr - arr.mean()

demeaned = people.groupby(key).transform(demean)
demeaned

Легко проверить, что в объекте demeaned групповые средние равны нулю:

In [ ]:
demeaned.groupby(key).mean()

## Метод аррlу

Как и aggregate, метод transform - более специализированная функция, предъявляющая жесткие требования: переданная ему функция должна возвращать либо скалярное значение, которое можно уложить (как np.mean), либо преобразованный массив такого же размера, что исходный. Самым общим из методов класса GroupBy является apply.

Apply разделяет обрабатываемый объект на части, вызывает для каждой части переданную функцию, а затем пытается конкатенировать все части вместе.

Возвращаясь к набору данных о чаевых, предположим, что требуется выбрать первые пять значений tip_pct в каждой группе. Прежде всего, нетрудно написать функцию, которая отбирает строки с наибольшими значениями в указанном столбце:

In [ ]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column) [-n:]

top(tips, n= 6)

Что здесь произошло? Функция **top** вызывается для каждой части DataFrame, после чего результаты склеиваются методом **pandas.concat**, а частям сопоставляются метки, совпадающие с именами групп. Поэтому результат имеет иерархический индекс, внутренний уровень которого содержит индексные значения из исходного объекта DataFrame. Если передать методу apply функцию, Которая принимает еще какие-то позиционные или именованные аргументы, то их можно передать вслед за самой функцией: 

In [ ]:
tips.groupby(['smoker', 'day']).apply(top, n = 1, column='total_bill')

In [ ]:
tips.groupby(['smoker', 'day']).apply(top, n = 1, column='total_bill')
result = tips.groupby('smoker')['tip_pct'].describe()
result

In [ ]:
result.unstack('smoker')

Когда от имени GroupBy вызывается метод типа describe, на самом деле выполняются такие предложения:

In [ ]:
f = lambda x: x.describe()
grouped.apply(f)

### Подавление групповых ключей

В примерах выше мы видели, что у результирующего объекта имеется иерархический индекс, образованный групповыми ключами и индексами каждой части исходного объекта. Создание этого индекса можно подавить, передав методу groupby параметр group_keys=False:

In [ ]:
tips.groupby('smoker', group_keys=False).apply(top)

# Упражнение

Федеральная избирательная комиссия США публикует данные о пожертвованиях участникам политических кампаний. Указывается имя жертвователя, род занятий, место работы, сумма пожертвования и т.п. 

https://classic.fec.gov/disclosurep/PDownload.do



In [ ]:
fec = read_csv('P00000001-ALL.csv')

In [ ]:
fec[:2]

In [ ]:
fec.iloc[123456]

#### Задание
Необходимо выдать сводную информацию в единой таблице по кандидатам:
- общая сумма пожертвований;
- количество городов, перечисливших наибольшую сумму за указанного кандидата.